# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 16 2023 8:30AM,255041,10,0086298099,ISDIN Corporation,Released
1,Jan 16 2023 8:30AM,255041,10,0086298108,ISDIN Corporation,Released
2,Jan 16 2023 8:30AM,255041,10,0086298132,ISDIN Corporation,Released
3,Jan 16 2023 8:30AM,255041,10,0086298180,ISDIN Corporation,Released
4,Jan 16 2023 8:30AM,255041,10,0086298366,ISDIN Corporation,Released
5,Jan 16 2023 8:30AM,255041,10,0086298377,ISDIN Corporation,Released
6,Jan 16 2023 8:30AM,255041,10,0086298376,ISDIN Corporation,Released
7,Jan 16 2023 8:30AM,255041,10,0086298375,ISDIN Corporation,Released
8,Jan 16 2023 8:30AM,255041,10,0086298362,ISDIN Corporation,Released
9,Jan 16 2023 8:30AM,255041,10,0086298378,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,255037,Released,1
15,255038,Released,6
16,255041,Released,15
17,255042,Released,7
18,255044,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
255037,NaN,NaN,1.0
255038,NaN,NaN,6.0
255041,NaN,NaN,15.0
255042,NaN,NaN,7.0
255044,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255015,0.0,0.0,1.0
255021,0.0,0.0,1.0
255026,0.0,1.0,20.0
255027,0.0,0.0,35.0
255028,1.0,7.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255015,0,0,1
255021,0,0,1
255026,0,1,20
255027,0,0,35
255028,1,7,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255015,0,0,1
1,255021,0,0,1
2,255026,0,1,20
3,255027,0,0,35
4,255028,1,7,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255015,,,1
1,255021,,,1
2,255026,,1,20
3,255027,,,35
4,255028,1,7,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 16 2023 8:30AM,255041,10,ISDIN Corporation
10,Jan 16 2023 8:30AM,255031,10,ISDIN Corporation
12,Jan 16 2023 8:30AM,255038,10,ISDIN Corporation
13,Jan 16 2023 8:30AM,255042,10,ISDIN Corporation
28,Jan 16 2023 8:30AM,255044,10,"Senseonics, Incorporated"
30,Jan 16 2023 8:30AM,255035,10,"Citieffe, Inc."
34,Jan 16 2023 8:30AM,255037,10,"Citieffe, Inc."
37,Jan 13 2023 3:47PM,255036,16,Sartorius Stedim Filters Inc.
38,Jan 13 2023 3:12PM,255033,19,"NAPP Technologies, LLC"
39,Jan 13 2023 2:52PM,255029,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 16 2023 8:30AM,255041,10,ISDIN Corporation,,,15
1,Jan 16 2023 8:30AM,255031,10,ISDIN Corporation,,,1
2,Jan 16 2023 8:30AM,255038,10,ISDIN Corporation,,,6
3,Jan 16 2023 8:30AM,255042,10,ISDIN Corporation,,,7
4,Jan 16 2023 8:30AM,255044,10,"Senseonics, Incorporated",,,2
5,Jan 16 2023 8:30AM,255035,10,"Citieffe, Inc.",,3,2
6,Jan 16 2023 8:30AM,255037,10,"Citieffe, Inc.",,,1
7,Jan 13 2023 3:47PM,255036,16,Sartorius Stedim Filters Inc.,,,1
8,Jan 13 2023 3:12PM,255033,19,"NAPP Technologies, LLC",,,1
9,Jan 13 2023 2:52PM,255029,10,ISDIN Corporation,,,29


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 16 2023 8:30AM,255041,10,ISDIN Corporation,15,,
1,Jan 16 2023 8:30AM,255031,10,ISDIN Corporation,1,,
2,Jan 16 2023 8:30AM,255038,10,ISDIN Corporation,6,,
3,Jan 16 2023 8:30AM,255042,10,ISDIN Corporation,7,,
4,Jan 16 2023 8:30AM,255044,10,"Senseonics, Incorporated",2,,
5,Jan 16 2023 8:30AM,255035,10,"Citieffe, Inc.",2,3,
6,Jan 16 2023 8:30AM,255037,10,"Citieffe, Inc.",1,,
7,Jan 13 2023 3:47PM,255036,16,Sartorius Stedim Filters Inc.,1,,
8,Jan 13 2023 3:12PM,255033,19,"NAPP Technologies, LLC",1,,
9,Jan 13 2023 2:52PM,255029,10,ISDIN Corporation,29,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 16 2023 8:30AM,255041,10,ISDIN Corporation,15,,
1,Jan 16 2023 8:30AM,255031,10,ISDIN Corporation,1,,
2,Jan 16 2023 8:30AM,255038,10,ISDIN Corporation,6,,
3,Jan 16 2023 8:30AM,255042,10,ISDIN Corporation,7,,
4,Jan 16 2023 8:30AM,255044,10,"Senseonics, Incorporated",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 16 2023 8:30AM,255041,10,ISDIN Corporation,15,NaN,NaN
1,Jan 16 2023 8:30AM,255031,10,ISDIN Corporation,1,NaN,NaN
2,Jan 16 2023 8:30AM,255038,10,ISDIN Corporation,6,NaN,NaN
3,Jan 16 2023 8:30AM,255042,10,ISDIN Corporation,7,NaN,NaN
4,Jan 16 2023 8:30AM,255044,10,"Senseonics, Incorporated",2,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 16 2023 8:30AM,255041,10,ISDIN Corporation,15,0.0,0.0
1,Jan 16 2023 8:30AM,255031,10,ISDIN Corporation,1,0.0,0.0
2,Jan 16 2023 8:30AM,255038,10,ISDIN Corporation,6,0.0,0.0
3,Jan 16 2023 8:30AM,255042,10,ISDIN Corporation,7,0.0,0.0
4,Jan 16 2023 8:30AM,255044,10,"Senseonics, Incorporated",2,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2295325,64,10.0,1.0
16,255036,1,0.0,0.0
18,255015,1,0.0,0.0
19,765081,37,0.0,0.0
20,255026,20,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2295325,64,10.0,1.0
1,16,255036,1,0.0,0.0
2,18,255015,1,0.0,0.0
3,19,765081,37,0.0,0.0
4,20,255026,20,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,64,10.0,1.0
1,16,1,0.0,0.0
2,18,1,0.0,0.0
3,19,37,0.0,0.0
4,20,20,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,64.0
1,16,Released,1.0
2,18,Released,1.0
3,19,Released,37.0
4,20,Released,20.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,19,20
Status,,,,,
Completed,1.0,0.0,0.0,0.0,0.0
Executing,10.0,0.0,0.0,0.0,1.0
Released,64.0,1.0,1.0,37.0,20.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,19,20
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,10.0,0.0,0.0,0.0,1.0
2,Released,64.0,1.0,1.0,37.0,20.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,19,20
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,10.0,0.0,0.0,0.0,1.0
2,Released,64.0,1.0,1.0,37.0,20.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()